**Set environment**

In [1]:
source ../config_duke.sh; show_env
source ../config_load_module_bedtools.sh

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect



## Set region
```
chr17: 8148000-8160000
```

**Set region by range**

In [2]:
CHROM=chr17
START=8148000
END=8160000

FP_OUT=${FD_RES}/sliding_window/region_PER1.bed
cat /dev/null > ${FP_OUT}
echo -e "${CHROM}\\t${START}\\t${END}" >> ${FP_OUT}

cat ${FP_OUT}

chr17	8148000	8160000


**Set region by base**

In [3]:
CHROM=chr17
START=8148000
END=8160000

FP_OUT=${FD_RES}/sliding_window/region_PER1_perbase.bed
cat /dev/null > ${FP_OUT}
for LOC in $(seq ${START} ${END}); do
    echo -e "${CHROM}\\t${LOC}\\t$((${LOC} + 1))" >> ${FP_OUT}
done

head ${FP_OUT}

chr17	8148000	8148001
chr17	8148001	8148002
chr17	8148002	8148003
chr17	8148003	8148004
chr17	8148004	8148005
chr17	8148005	8148006
chr17	8148006	8148007
chr17	8148007	8148008
chr17	8148008	8148009
chr17	8148009	8148010


## get the coverage of sliding window

**test loop**

In [5]:
WINDOW=sliding_chr17_len_1kb_gap_100bp
FD_WIN=${FD_RES}/sliding_window

ls ${FD_WIN}/${WINDOW}

dat_mu_var.tsv  Input3      Input5_20x            TFX3_Dex   TFX5_DMSO
Input1          Input3_20x  lowess_input_20x.rds  TFX3_DMSO
Input1_20x      Input4      lowess_tfx_dmso.rds   TFX4_Dex
Input2          Input4_20x  TFX2_Dex              TFX4_DMSO
Input2_20x      Input5      TFX2_DMSO             TFX5_Dex


In [8]:
WINDOW=sliding_chr17_len_1kb_gap_100bp
FD_WIN=${FD_RES}/sliding_window

FD_BEDS=($(ls -d ${FD_WIN}/${WINDOW}/{Input?*,TFX?_DMSO,TFX?_Dex}/))
#FD_BED=${FD_BEDS[${SLURM_ARRAY_TASK_ID}]}
FN_BED=chr17_count.bed.gz

for FD_BED in ${FD_BEDS[@]}; do
    ls ${FD_BED}/${FN_BED}
done

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/Input1//chr17_count.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/Input1_20x//chr17_count.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/Input2//chr17_count.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/Input2_20x//chr17_count.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/Input3//chr17_count.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/Input3_20x//chr17_count.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/Input4//chr17_count.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/Input4_20x//chr17_count.bed.gz


In [9]:
ls -d ${FD_WIN}/${WINDOW}/{Input?*,TFX?_DMSO,TFX?_Dex}/ | wc -l

18


In [10]:
ls ${FD_WIN}

region_PER1.bed                      sliding_chr17_len_1kb_gap_100bp_f50_OR_F50
region_PER1_perbase.bed              sliding_PER1_len_1kb_gap_100bp
sliding_chr17_len_1kb_gap_100bp      sliding_PER1_len_1kb_gap_100bp.bed
sliding_chr17_len_1kb_gap_100bp.bed


**Run**

In [23]:
sbatch -p ${NODE} \
    --array 0-17 \
    --mem 10G \
    -o ${FD_LOG}/coverage_sliding_windows_PER1.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
CHROM=chr17

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set file IO
WINDOW=sliding_chr17_len_1kb_gap_100bp
FD_WIN=${FD_RES}/sliding_window
FN_REG=region_PER1_perbase.bed

FD_BEDS=($(ls -d ${FD_WIN}/${WINDOW}/{Input?*,TFX?_DMSO,TFX?_Dex}/))
FD_BED=${FD_BEDS[${SLURM_ARRAY_TASK_ID}]}
FN_BED=chr17_count.bed.gz
FN_OUT=target_PER1_coverage.bed.gz

### print input Message
echo ++++++++++++++++++++++++++++++++++++++++++++++++
echo "SAMPLE:      " ${SAMPLE}
echo "Input  file: " ${FD_BED}/${FN_BED}
echo "Input  file: " ${FD_WIN}/${FN_REG}
echo "Output file: " ${FD_BED}/${FN_OUT}
echo
echo "Show the first few lines of the input file"
echo ${FD_BED}/${FN_BED}
zcat ${FD_BED}/${FN_BED} | head -n 3 
echo
echo ${FD_WIN}/${FN_REG}
cat  ${FD_WIN}/${FN_REG} | head -n 3 
echo

### run: get the coverage of sliding window
bedtools map \
    -a ${FD_WIN}/${FN_REG} \
    -b ${FD_BED}/${FN_BED} \
    -c 4 \
    -o sum |\
    gzip -c \
    > ${FD_BED}/${FN_OUT}

### print output Message
echo "Show the first few lines of the output file"
echo ${FD_BED}/${FN_OUT}
zcat ${FD_BED}/${FN_OUT} | head -n 3
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 28684348


In [24]:
cat ${FD_LOG}/coverage_sliding_windows_PER1.0.txt

Slurm Array Index:  0
Time Stamp:         07-26-22+23:44:54

++++++++++++++++++++++++++++++++++++++++++++++++
SAMPLE:      
Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/Input1//chr17_count.bed.gz
Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/region_PER1_perbase.bed
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/Input1//target_PER1_coverage.bed.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/Input1//chr17_count.bed.gz
chr17	200200	201200	2
chr17	200300	201300	4
chr17	200400	201400	5

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/region_PER1_perbase.bed
chr17	8148000	8148001
chr17	8148001	8148002
chr17	8148002	8148003

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_w

In [25]:
cat ${FD_LOG}/coverage_sliding_windows_PER1.10.txt

Slurm Array Index:  10
Time Stamp:         07-26-22+23:44:54

++++++++++++++++++++++++++++++++++++++++++++++++
SAMPLE:      
Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/TFX2_Dex//chr17_count.bed.gz
Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/region_PER1_perbase.bed
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/TFX2_Dex//target_PER1_coverage.bed.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/TFX2_Dex//chr17_count.bed.gz
chr17	82700	83700	1
chr17	82800	83800	1
chr17	82900	83900	1

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/region_PER1_perbase.bed
chr17	8148000	8148001
chr17	8148001	8148002
chr17	8148002	8148003

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_

In [26]:
sbatch -p ${NODE} \
    --array 0-17 \
    --mem 10G \
    -o ${FD_LOG}/coverage_sliding_windows_f50_PER1.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
CHROM=chr17

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set file IO
WINDOW=sliding_chr17_len_1kb_gap_100bp_f50_OR_F50
FD_WIN=${FD_RES}/sliding_window
FN_REG=region_PER1_perbase.bed

FD_BEDS=($(ls -d ${FD_WIN}/${WINDOW}/{Input?*,TFX?_DMSO,TFX?_Dex}/))
FD_BED=${FD_BEDS[${SLURM_ARRAY_TASK_ID}]}
FN_BED=chr17_count.bed.gz
FN_OUT=target_PER1_coverage.bed.gz

### print input Message
echo ++++++++++++++++++++++++++++++++++++++++++++++++
echo "SAMPLE:      " ${SAMPLE}
echo "Input  file: " ${FD_BED}/${FN_BED}
echo "Input  file: " ${FD_WIN}/${FN_REG}
echo "Output file: " ${FD_BED}/${FN_OUT}
echo
echo "Show the first few lines of the input file"
echo ${FD_BED}/${FN_BED}
zcat ${FD_BED}/${FN_BED} | head -n 3 
echo
echo ${FD_WIN}/${FN_REG}
cat  ${FD_WIN}/${FN_REG} | head -n 3 
echo

### run: get the coverage of sliding window
bedtools map \
    -a ${FD_WIN}/${FN_REG} \
    -b ${FD_BED}/${FN_BED} \
    -c 4 \
    -o sum |\
    gzip -c \
    > ${FD_BED}/${FN_OUT}

### print output Message
echo "Show the first few lines of the output file"
echo ${FD_BED}/${FN_OUT}
zcat ${FD_BED}/${FN_OUT} | head -n 3
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 28684366
